In [2]:
import requests
import json
import random

In [3]:
import requests
import json

# --- Configuration ---
BASE_API_URL = "http://127.0.0.1:8000/api/v1/" # Adjust if your server runs elsewhere
ACCOMMODATIONS_ENDPOINT = f"{BASE_API_URL}accommodations/"
MEMBERS_ENDPOINT = f"{BASE_API_URL}members/"
RESERVATIONS_ENDPOINT = f"{BASE_API_URL}reservations/"

# --- Data ---
# (Converted from your text format to Python lists of dictionaries)

accommodations_data = [
    {
        "original_ref": "#1", # Helper field for linking reservations later
        "room_number": 1,
        "flat_number": "C",
        "floor_number": 3,
        "building_name": "Jolly Villa",
        "availability_start": "2025-03-01",
        "availability_end": "2025-08-31",
        "managed_by": "HKU"
    },
    {
        "original_ref": "#2",
        "room_number": None, # Handle (null) as None
        "flat_number": "G",
        "floor_number": 22,
        "building_name": "South View Garden",
        "availability_start": "2025-04-01",
        "availability_end": "2025-10-31",
        "managed_by": "HKU"
    },
    {
        "original_ref": "#3",
        "room_number": 3,
        "flat_number": "E",
        "floor_number": 12,
        "building_name": "Glen Haven",
        "availability_start": "2025-01-01",
        "availability_end": "2025-12-31",
        "managed_by": "HKU, CUHK" # Assuming CharField can hold this
    },
    {
        "original_ref": "#4",
        "room_number": None,
        "flat_number": "D",
        "floor_number": 2,
        "building_name": "Prosperity Mansion",
        "availability_start": "2025-03-15",
        "availability_end": "2025-07-31",
        "managed_by": "CHUK" # Corrected based on input
    }
]

members_data = [
    {"name": "Anson Lee", "contact": "2290 4324", "institute": "HKU"},
    {"name": "Candy Chan", "contact": "3528 6925", "institute": "HKU"},
    {"name": "Billy Johnson", "contact": "3910 1481", "institute": "CUHK"},
    {"name": "Fred Lam", "contact": "3859 4679", "institute": "HKU"}
]

# Reservations data needs IDs from created accommodations and members
reservations_raw_data = [
    {"accommodation_ref": "#2", "start_date": "2025-04-15", "end_date": "2025-04-21", "member_name": "Anson Lee", "status_text": "Contract Signed"},
    {"accommodation_ref": "#1", "start_date": "2025-04-22", "end_date": "2025-05-14", "member_name": "Anson Lee", "status_text": "Contract Signed"},
    {"accommodation_ref": "#1", "start_date": "2025-06-15", "end_date": "2025-06-30", "member_name": "Anson Lee", "status_text": "Contract Not signed"},
    {"accommodation_ref": "#3", "start_date": "2025-05-22", "end_date": "2025-07-07", "member_name": "Candy Chan", "status_text": "Contract Signed"},
    {"accommodation_ref": "#3", "start_date": "2025-03-01", "end_date": "2025-05-07", "member_name": "Billy Johnson", "status_text": "Contract Signed"}
]

# --- Helper Function ---
def post_data(endpoint, data):
    """Posts a single item to the specified endpoint."""
    try:
        response = requests.post(endpoint, json=data)
        response.raise_for_status() # Raise an exception for bad status codes (4xx or 5xx)
        print(f"SUCCESS: Posted to {endpoint}. ID: {response.json().get('id')}")
        return response.json() # Return the created object data (includes ID)
    except requests.exceptions.RequestException as e:
        print(f"ERROR posting to {endpoint}: {e}")
        if hasattr(e, 'response') and e.response is not None:
            try:
                print(f"Response body: {e.response.json()}")
            except json.JSONDecodeError:
                print(f"Response body: {e.response.text}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

# --- Main Execution ---

# Dictionaries to store the IDs of created objects
created_accommodation_ids = {} # Maps original_ref -> created_id
created_member_ids = {}      # Maps name -> created_id

print("--- Posting Accommodations ---")
for acc_data in accommodations_data:
    original_ref = acc_data.pop("original_ref") # Remove helper field before posting
    created_acc = post_data(ACCOMMODATIONS_ENDPOINT, acc_data)
    if created_acc and 'id' in created_acc:
        created_accommodation_ids[original_ref] = created_acc['id']

print("\n--- Posting Members ---")
for mem_data in members_data:
    created_mem = post_data(MEMBERS_ENDPOINT, mem_data)
    if created_mem and 'id' in created_mem:
        # Using name as key, assuming names are unique in this sample data
        created_member_ids[mem_data['name']] = created_mem['id']

print("\n--- Posting Reservations ---")
if not created_accommodation_ids or not created_member_ids:
    print("ERROR: Cannot post reservations because accommodation or member posting failed.")
else:
    for res_raw in reservations_raw_data:
        # Map status text to model choices key
        status_map = {
            "Contract Signed": "Signed",
            "Contract Not signed": "Not Signed"
        }
        status_key = status_map.get(res_raw["status_text"])

        # Look up the actual IDs
        acc_id = created_accommodation_ids.get(res_raw["accommodation_ref"])
        mem_id = created_member_ids.get(res_raw["member_name"])

        if not acc_id:
            print(f"ERROR: Could not find created ID for accommodation ref: {res_raw['accommodation_ref']}. Skipping reservation.")
            continue
        if not mem_id:
            print(f"ERROR: Could not find created ID for member name: {res_raw['member_name']}. Skipping reservation.")
            continue
        if not status_key:
             print(f"ERROR: Invalid status text: {res_raw['status_text']}. Skipping reservation.")
             continue

        # Prepare final reservation data with IDs
        reservation_data = {
            "accommodation": acc_id,
            "member": mem_id,
            "start_date": res_raw["start_date"],
            "end_date": res_raw["end_date"],
            "status": status_key
        }

        post_data(RESERVATIONS_ENDPOINT, reservation_data)

print("\n--- Data Posting Complete ---")

--- Posting Accommodations ---
SUCCESS: Posted to http://127.0.0.1:8000/api/v1/accommodations/. ID: 1
SUCCESS: Posted to http://127.0.0.1:8000/api/v1/accommodations/. ID: 2
SUCCESS: Posted to http://127.0.0.1:8000/api/v1/accommodations/. ID: 3
SUCCESS: Posted to http://127.0.0.1:8000/api/v1/accommodations/. ID: 4

--- Posting Members ---
SUCCESS: Posted to http://127.0.0.1:8000/api/v1/members/. ID: 1
SUCCESS: Posted to http://127.0.0.1:8000/api/v1/members/. ID: 2
SUCCESS: Posted to http://127.0.0.1:8000/api/v1/members/. ID: 3
SUCCESS: Posted to http://127.0.0.1:8000/api/v1/members/. ID: 4

--- Posting Reservations ---
SUCCESS: Posted to http://127.0.0.1:8000/api/v1/reservations/. ID: 1
SUCCESS: Posted to http://127.0.0.1:8000/api/v1/reservations/. ID: 2
SUCCESS: Posted to http://127.0.0.1:8000/api/v1/reservations/. ID: 3
SUCCESS: Posted to http://127.0.0.1:8000/api/v1/reservations/. ID: 4
SUCCESS: Posted to http://127.0.0.1:8000/api/v1/reservations/. ID: 5

--- Data Posting Complete --